In [1]:
from tyssue.geometry import base_geometry, bulk_geometry, cellcell_geometry, planar_geometry, sheet_geometry

In [2]:
%load_ext autoreload
%autoreload 2

### 
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi

import vispy as vp
from vispy import app, gloo, visuals, scene


import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry.bulk_geometry import BulkGeometry as bgeom
from tyssue.core.generation import hexa_grid3d, from_3d_voronoi


### Scatter plots of the cell centers

```python
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
```

In [6]:
grid = hexa_grid3d(16, 14, 3)

datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

gspecs = eptm.set_geom('sheet')
bgeom.update_all(eptm)
eptm.cell_df.head()

,x,num_faces,y,z,vol,is_alive
cell,,,,,,
0,0.75,4,0.649519,0.433013,-3.338279e-33,True
1,1.50,8,0.649519,0.433013,2.083333e-02,True
2,2.50,8,0.649519,0.433013,2.083333e-02,True
3,3.50,8,0.649519,0.433013,2.083333e-02,True
4,4.50,8,0.649519,0.433013,2.083333e-02,True


In [4]:
bounds = [[-0.1, 2], [0, 2], [0, 10]]

je_out = eptm.cut_out(bounds)

eptm.remove(je_out)

In [12]:
vertices, faces, face_mask = eptm.triangular_mesh(eptm.coords)

canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1


view.bgcolor = vp.color.Color('#aaaaaa')



mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces, color=[0.2, 0.5, 0.1, 0.8])

wire_pos = vertices[eptm.Nc:].copy()


wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2] - eptm.Nc,
                             color=[0.1, 0.1, 0.3, 0.8],
                             width=1)
ccenters = vp.scene.visuals.Markers(
        pos=eptm.cell_df[eptm.coords].values,
        face_color=[1, 1, 1])

#view.add(mesh)
view.add(wire)
# view.add(ccenters)
canvas.show()

app.run()

0

In [8]:
eptm.jv_df['z'].ptp()

1.5155444566227692

In [10]:
eptm.jv_df['z'] = eptm.jv_df['z'] * np.cos(eptm.jv_df['y'])
bgeom.update_all(eptm)

In [6]:
import vispy.io as io

In [7]:
bgeom.update_all(eptm)

In [8]:
eptm.je_df.head()

,dx,srce,dy,dz,nx,ny,length,face,trgt,nz,cell,sub_area,sub_vol
je,,,,,,,,,,,,,
0,-0.5,10,0.288675,0.144338,0.0625,0.054127,0.595119,14,6,0.108253,0,0.068108,-7.703720e-34
1,0.0,6,-0.433013,0.216506,0.0625,0.054127,0.484123,14,4,0.108253,0,0.068108,-5.135813e-34
2,0.5,4,-0.288675,-0.144338,0.0625,0.054127,0.595119,14,11,0.108253,0,0.068108,-8.987673e-34
3,0.0,11,0.433013,-0.216506,0.0625,0.054127,0.484123,14,10,0.108253,0,0.068108,-1.155558e-33
4,0.0,16,-0.433013,0.216506,0.0625,0.054127,0.484123,4,12,0.108253,1,0.068108,2.604167e-03


In [9]:
io.write_mesh('test.obj', vertices=vertices, faces=faces,
              normals=eptm.je_df[['nx', 'ny', 'nz']].values,
              texcoords=None, overwrite=True)